In [1]:
# import libraries

# for processing data tables
import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for sending requests to url and parsing
import requests
import json
import xmltodict

In [2]:
start_year = 2015
end_year = 2020

## street_density

In [84]:
def get_street_density(base_url, start_year, end_year):
    
    # create lists to store annual information
    list_metro = []
    list_basic = []

    for year in range(2015,2021):
        # url for total and street area of local region // this should be from a database or csv file as a variable
        url = base_url.format(year,year)

        # convert a json formatted response to a Pandas Dataframe
        response = requests.get(base_url.format(year,year))

        # divide the dataframe based on 'C3_NM'
        # C3_NM == total (계)
        df_area = pd.DataFrame(json.loads(response.text))
        df_total_area =  df_total.loc[df_total['C3_NM']=='계'].loc[:, ['C1_NM', 'C2_NM', 'DT']].rename(columns = {'DT':'total_area_m2'})
        df_total_area['total_area_m2'] = df_total_area['total_area_m2'].astype(float)
        df_total_area_metro = df_total_area.loc[df_total_area['C1_NM']==df_total_area['C2_NM'],:].drop('C2_NM', axis = 1)
        df_total_area_basic = df_total_area.loc[df_total_area['C1_NM']!=df_total_area['C2_NM'],:]

        # C3_NM == road (도로)
        df_road_area = df_total.loc[df_total['C3_NM']=='도로'].loc[:, ['C1_NM', 'C2_NM', 'DT']].rename(columns = {'DT':'road_area_m2'})
        df_road_area['road_area_m2'] = df_road_area['road_area_m2'].astype(float)
        df_road_area_metro = df_road_area.loc[df_road_area['C1_NM']==df_road_area['C2_NM'],:].drop('C2_NM', axis = 1)
        df_road_area_basic = df_road_area.loc[df_road_area['C1_NM']!=df_road_area['C2_NM'],:]

        # merge total and road dataframe
        df_metro = df_total_area_metro.merge(df_road_area_metro, on ='C1_NM')
        df_basic = df_total_area_basic.merge(df_road_area_basic, on =['C1_NM', 'C2_NM'])

        # add `year` column
        df_metro['year'] = year
        df_basic['year'] = year

        # append the dataframes into the list
        list_metro.append(df_metro)
        list_basic.append(df_basic)

    # concatenate the annual metro region dataframes into one dataframe    
    df_metro_agg = pd.concat(list_metro, ignore_index = False)
    # calculate the street density
    df_metro_agg['street_density'] = df_metro_agg['road_area_m2'] / df_metro_agg['total_area_m2']
    
    # concatenate the annual basic region dataframes into one dataframe    
    df_basic_agg = pd.concat(list_basic, ignore_index = False)
    # calculate the street density
    df_basic_agg['street_density'] = df_basic_agg['road_area_m2'] / df_basic_agg['total_area_m2']
    
    return df_metro_agg, df_basic_agg

In [85]:
df_metro, df_basic = get_street_density(base_url, start_year, end_year)

In [86]:
df_metro.head()

,C1_NM,total_area_m2,road_area_m2,year,street_density
0,전국,1.004126e+11,3.386296e+09,2015,0.033724
1,서울,6.052285e+08,7.954328e+07,2015,0.131427
2,부산,7.700748e+08,5.896284e+07,2015,0.076568
3,대구,8.834875e+08,5.956761e+07,2015,0.067423
4,인천,1.065226e+09,8.399425e+07,2015,0.078851


In [87]:
df_basic.head()

,C1_NM,C2_NM,total_area_m2,road_area_m2,year,street_density
0,서울,종로구,23913168.3,2371018.6,2015,0.099151
1,서울,중구,9960304.5,2147045.1,2015,0.215560
2,서울,용산구,21866144.1,2481423.2,2015,0.113482
3,서울,성동구,16861003.9,2266727.1,2015,0.134436
4,서울,광진구,17062938.6,2498767.7,2015,0.146444


## land_allocated_to_streets